In [38]:
#PLOTS W GAPS a

import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import gsw

#Function that calculates buoyancy frequency
def calculate_buoy_freq(sals, temps, depths):
    lat = np.repeat(59.153,np.shape(sals)[0])
    lon = np.repeat(-58.895,np.shape(sals)[0])
    abs_sal = gsw.SA_from_SP(sals,depths,lat,lon)
    pot_temp = gsw.pt0_from_t(abs_sal, temps, depths)
    cons_temp = gsw.conversions.CT_from_pt(abs_sal,pot_temp)
    density = gsw.density.sigma0(abs_sal,cons_temp)
    buoy_freq = gsw.Nsquared(abs_sal, cons_temp, depths, lat)
    return buoy_freq[0]

numb_years = 17

letters = ['a', 'b', 'd']
years = ['0304','0405','0506','0708','0910','1011','1011', '1213','1314','1415','1516','1617', '1718','1718','1821','2122','2122']
files = [314, 322, 330, 297, 324, 270, 764, 328, 366, 330, 326, 310, 2707, 325, 949, 708, 708] #prob last one - 6, 13, 14

number_file = np.sum(files[0:numb_years])
# empty_array = np.empty([971*number_file,8])
dts = np.empty([971*number_file], dtype='U25')
depths = np.empty([971*number_file])
sals = np.empty([971*number_file])
temps = np.empty([971*number_file])
us = np.empty([971*number_file])
vs = np.empty([971*number_file])
ws = np.empty([971*number_file])
buoys = np.empty([number_file])
u_maxs = np.empty([number_file])
u_max_uppers = np.empty([number_file])
dt_buoy = np.empty([number_file], dtype='U25')
dt_depth = np.empty([number_file])

dt_depth[:]=np.nan
buoys[:]=np.nan
# dt_buoy = np.nan

# print(np.shape(empty_array))
index = 0
index2 = 0

for j in range(0,1):
    #Start with the first file
    path = '/Users/helena/Downloads/bg' + years[0] + '_mmp_' + letters[j] + '/bg' + years[0] + '_mmp_' + letters[j] + '001.dat'
    data = np.loadtxt(path, dtype = str, skiprows = 2)
    header = np.loadtxt(path, dtype = str, max_rows=1)
    
    #This loops through each year of data
    for k in range(0,numb_years):
        print(k)
        
        #This loop creates the file names of each file in the year's folder 
        profile_names = np.zeros((files[k],1), dtype=object)
        for i in range(1, files[k]+1):
            #The conditionals are to account for the years that had two folders of data
            if k==1 and j==1:
                name = 'bg' + years[k] + '_mmp_' + letters[j] + '{0:0>4}'.format(i) + '.dat'
            elif k==14 and j==1:
                name = 'bg' + years[k] + '_mmp_' + letters[j] + '-2-' + '{0:0>3}'.format(i) + '.dat'
            elif k==6 and j==0:
                name = 'bg' + years[k] + '_mmp_' + letters[j] + '-2-' + '{0:0>3}'.format(i) + '.dat'
            elif k==13 and j==0:
                name = 'bg' + years[k] + '_mmp_' + letters[j] + '-2-' + '{0:0>3}'.format(i) + '.dat'
            elif k==14 and j==0:
                name = 'bg' + years[k] + '_mmp_' + letters[j] + '-2-' + '{0:0>3}'.format(i) + '.dat'
            elif k==16 and j==0:
                name = 'bg' + years[k] + '_mmp_' + letters[j] + '-2-' + '{0:0>3}'.format(i) + '.dat'
            else:
                name = 'bg' + years[k] + '_mmp_' + letters[j] + '{0:0>3}'.format(i) + '.dat'
            profile_names[i-1] = name
        
        #This loop saves the data from each file in the folder
        for i in range(0, files[k]):
            try: 
                #These were the corrupted data files
                if profile_names[i][0] == 'bg0910_mmp_b163.dat':
                    raise IOError
                if profile_names[i][0] == 'bg0910_mmp_b171.dat':
                    raise IOError
                if profile_names[i][0] == 'bg0910_mmp_b187.dat':
                    raise IOError
                
                #The conditionals are to account for the years that had two folders of data 
                if k==14 and j==1:
                    path = '/Users/helena/Downloads/bg' + years[k] + '_mmp_' + letters[j] + '-2' + '/' + profile_names[i][0]
                elif k==6 and j==0:
                    path = '/Users/helena/Downloads/bg' + years[k] + '_mmp_' + letters[j] + '-2' + '/' + profile_names[i][0]
                elif k==13 and j==0:
                    path = '/Users/helena/Downloads/bg' + years[k] + '_mmp_' + letters[j] + '-2' + '/' + profile_names[i][0]
                elif k==14 and j==0:
                    path = '/Users/helena/Downloads/bg' + years[k] + '_mmp_' + letters[j] + '-2' + '/' + profile_names[i][0]
                elif k==16 and j==0:
                    path = '/Users/helena/Downloads/bg' + years[k] + '_mmp_' + letters[j] + '-2' + '/' + profile_names[i][0]
                else:
                    path = '/Users/helena/Downloads/bg' + years[k] + '_mmp_' + letters[j] + '/' + profile_names[i][0]
                
                header = np.loadtxt(path, dtype = str, max_rows=1)
                data = np.loadtxt(path, dtype = str, skiprows = 2)
                
                date = header[6] + ' ' + data[0][0]
                
                
                #Saving the data
                
                #The data from deep MMPs contained measurements at odd and even depth intervals, so they needed to be processed in order to match up with the rest of the data
                if k==6 or k==14 or k==15 or k==16:
                    mask = [True if float(i)%2!=0 else False for i in data[:,1]]
                    dt = np.repeat(date,np.shape(data[mask])[0])
                    numm = np.shape(data[mask])[0]

                    dts[index:index+numm] = dt
                    depths[index:index+numm] = data[:,1][mask]
                    sals[index:index+numm] = data[:,3][mask]
                    temps[index:index+numm] = data[:,2][mask]
                    us[index:index+numm] = data[:,4][mask]
                    vs[index:index+numm] = data[:,5][mask]
                    ws[index:index+numm] = data[:,6][mask]
                    
                    #** This code calculates max buoyancy frequency and max u velocity for the file
                    
                    masky = data[:,1][mask].astype(float)<101
                    buoy_freq_temp = calculate_buoy_freq(data[:,3][mask][masky],data[:,2][mask][masky],data[:,1][mask][masky])
                    
                    if buoy_freq_temp.size and np.isnan(buoy_freq_temp).all()==False:
                        buoys[index2] = np.nanmax(buoy_freq_temp)
                        ind = np.nanargmax(buoy_freq_temp)
                        dt_depth[index2] = data[:,1][mask][masky][ind+1]
                        dt_buoy[index2] = date
                        
                    if us[index:index+numm].size:
                        u_maxs[index2] = np.nanmax(us)
                    
                    index2 = index2+1
                    #**
                    
                    index = np.shape(data[mask])[0] + index
                
                #These are the data from the shallow MMPs
                else:
                    dt = np.repeat(date,np.shape(data)[0])
                    numm = np.shape(data)[0]

                    dts[index:index+numm] = dt
                    depths[index:index+numm] = data[:,1]
                    sals[index:index+numm] = data[:,3]
                    temps[index:index+numm] = data[:,2]
                    us[index:index+numm] = data[:,4]
                    vs[index:index+numm] = data[:,5]
                    ws[index:index+numm] = data[:,6]
                    
                    #**
                    masky = data[:,1].astype(float)<101
                    buoy_freq_temp = calculate_buoy_freq(data[:,3][masky],data[:,2][masky],data[:,1][masky])

                    if buoy_freq_temp.size and np.isnan(buoy_freq_temp).all()==False:
                        buoys[index2] = np.nanmax(buoy_freq_temp)
                        ind = np.nanargmax(buoy_freq_temp)
                        dt_depth[index2] = data[:,1][masky][ind+1]
                        dt_buoy[index2] = date
                        
                    if us[index:index+numm].size:
                        u_maxs[index2] = np.nanmax(us)
                    index2 = index2 + 1
                    #**

                    index = np.shape(data)[0] + index
                

            except IOError:
                0 #This clause lets the program skip over missing profiles in the data folder
            except IndentationError:
                print("hopefully nothing happens")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [41]:
#Salinity Graph

fig = go.Figure(data=go.Heatmap(
                    z=sals,
                    x=dts,
                    y=depths,
                    colorscale='Viridis',
                    colorbar_title = 'psu', zmin = 30, zmax = 35))
fig.update_yaxes(autorange="reversed",range = [0,2000])
fig.update_layout(title_text="Salinity - Mooring A")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Depth [m]",
    font=dict(size=30))
fig.write_html("/Users/helena/Downloads/salinity_a.html")

In [39]:
#Temperature Graph

fig = go.Figure(data=go.Heatmap(
                    z=temps,
                    x=dts,
                    y=depths,
                    colorscale='Viridis', 
                    colorbar_title = '°C'),
                layout_yaxis_range=[0,2000])
fig.update_yaxes(autorange="reversed")
fig.update_layout(title_text="Temperature - Mooring A")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Depth [m]",
    font=dict(size=30))
fig.write_html("/Users/helena/Downloads/temperature_a.html")

In [43]:
#Speed Graph

speeds = np.sqrt(np.square(us) + np.square(vs))
fig = go.Figure(data=go.Heatmap(
                    z=speeds,
                    x=dts,
                    y=depths,
                    colorscale='Viridis',
                    colorbar_title = 'cm/s'))
fig.update_yaxes(autorange="reversed",range = [0,2000])
fig.update_layout(title_text="Speed - Mooring A")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Depth [m]",
    font=dict(size=30))
fig.write_html("/Users/helena/Downloads/speeds_a.html")

In [44]:
#Buoyancy Frequency Graph

#Calculates the buoyancy frequency array 
import gsw
lat = np.repeat(59.153,np.shape(sals)[0])
lon = np.repeat(-58.895,np.shape(sals)[0])

abs_sal = gsw.SA_from_SP(sals,depths,lat,lon)
pot_temp = gsw.pt0_from_t(abs_sal, temps, depths)
cons_temp = gsw.conversions.CT_from_pt(abs_sal,pot_temp)
density = gsw.density.sigma0(abs_sal,cons_temp)
buoy_freq = gsw.Nsquared(abs_sal, cons_temp, depths, lat)

fig = go.Figure(data=go.Heatmap(
                    z=buoy_freq[0],
                    x=dts,
                    y=depths,
                    colorscale='Viridis', 
                    colorbar_title = '1/s', zmin = 0, zmax = .0003))
fig.update_yaxes(autorange="reversed",range = [0,2000])
fig.update_layout(title_text="Buoyancy Frequency - Mooring A")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Depth [m]",
    font=dict(size=30))
fig.write_html("/Users/helena/Downloads/buoy_freq_a.html")

/Users/helena/anaconda3/lib/python3.8/site-packages/gsw/stability.py:86: RuntimeWarning:

divide by zero encountered in true_divide

/Users/helena/anaconda3/lib/python3.8/site-packages/gsw/stability.py:87: RuntimeWarning:

invalid value encountered in multiply



In [45]:
#Plots the Buoyancy Frequency with a line tracing its max value

heatmap = go.Heatmap(z=buoy_freq[0],x=dts,y=depths, zmin = 0, zmax = .0003, colorscale='Viridis', colorbar_title = '1/s')
lines = go.Scatter(x=dt_buoy, y = dt_depth, mode='lines', line_color='black', line_width=1)
fig = go.Figure(data=[heatmap, lines])
fig.update_yaxes(autorange="reversed")
fig.update_layout(title_text="Buoyancy Frequency and Max Line - Mooring A")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Depth [m]",
    font=dict(size=30))
fig.write_html("/Users/helena/Downloads/buoy_freq_line_a.html")